In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path, set_mpl
from scripts.cross_match_scripts import fits_to_pandas, pandas_to_fits, decode_str_columns
import pandas as pd
from tqdm import tqdm
import numpy as np
import seaborn as sns
from astropy.coordinates import SkyCoord
import astropy.coordinates 
import astropy.units as u
import astroquery

%matplotlib inline

load DESI counterparts

In [3]:
desi = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14_desi_nway_match.gz_pkl', compression='gzip')
desi.reset_index(drop=True, inplace=True)
desi[['nway_desi_id', 'desi_ra', 'desi_dec']].to_csv(data_path+'desi_LH_coords.csv', index=False)
desi.head(10)

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2,nway_nnmag_grzw1,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_closest,nway_is_within_pos_r98,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id
0,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,0.006185,7.077317,154.763934,56.505585,853324209744036352,7.576342,1,0,20.950485,-1.0,0,SDSS J101902.92+563016.4,0.949746,QSO,7.576898,1,0,0,2,16.915001,16.554001,12.382,12.1,5.0,0.7,2,1,1,1237658303814435106,9216733310671409152,7.544174,68469189470,4.058504,3.313348,5.001804,8.110237,132.352465,42.185604,1.670761e-14,1.526396e+20,9011_606779_231,5.167984,9.665420e-01,0.976823,0.942008,0.930541,0.877164,14,0,False,True,9011,231,606779,154.762154,56.504538,3.362170,4.618722,5.989522,16.031805,20.984790,50.180990,-3.493270,435.342250,248.158630,104.400180,3.761635,0.939089,0.001432,0.000018,21.087423,20.774010,20.520338,19.482046,19.191862,18.247932,NaN,70.151200,72.758970,61.198788,31.093582,20.335644,1.898610,-0.014626,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0
1,SRGe J101903.4+563325,154.764302,56.556994,154.897407,50.039645,12.232697,16.660557,2.758328e-14,6.850312e-15,34.209091,8.495835,1133.034668,0.0,0.0,0.0,4806,3902,154.766311,56.555705,5.407337,163.025894,0.000029,6.268413e+08,6.270701e+08,0.0,0.0,0.030192,0.007498,6.562579,154.764302,56.556994,-1,-1.000000,0,-1,NaN,-1.0,-1,SDSS J101904.67+563333.2,1.464000,QSO,13.943999,1,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,68265812198,5.407337,4.373272,6.601858,10.704663,132.318989,42.230639,2.122641e-14,1.507868e+20,9011_606779_284,12.929691,4.530361e-01,0.835886,0.953387,0.928812,0.577701,9,0,False,False,9011,284,606779,154.769507,56.559155,4.110681,6.741001,7.775545,18.952187,29.332830,65.296720,371.384800,259.819270,99.752320,72.978060,3.654050,0.866400,0.001278,0.000015,20.872555,20.365770,20.238258,19.300547,18.828358,17.962076,16.075178,66.259700,67.326480,66.424300,36.228188,27.303180,2.334288,1.453649,-0.009351,1.285011,-0.184517,0.341205,-2.763030,0.601020,G2,853371518808768256
2,SRGe J101911.0+562639,154.795822,56.444182,155.033709,50.112976,7.855049,18.953989,1.654359e-14,3.709296e-15,40.391277,9.056271,2230.516357,0.0,0.0,0.0,4530,3655,154.797830,56.442893,3.409649,142.069946,0.000015,6.268413e+08,6.270701e+08,0.0,0.0,0.018108,0.004060,12.856621,154.795822,56.444182,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,68713641563,3.409649,2.808233,4.239287,6.873844,132.413453,42.140474,1.273093e-14,1.537238e+20,9011_606779_455,15.145586,5.746393e-07,0.994162,-99.000000,-99.000000,0.230064,7,0,True,False,9011,455,606779,154.798428,56.448135,0.261824,0.279961

# XMATCH

http://cdsxmatch.u-strasbg.fr/

## with GAIA

https://vizier.cfa.harvard.edu/viz-bin/VizieR-3?-source=I/355/gaiadr3

In [4]:
xmatch_gaia = pd.read_csv(data_path+'desi_LH_coords_gaiadr3_large_r.csv')
print(len(xmatch_gaia), 'gaia sources after cross-match')
xmatch_gaia = xmatch_gaia.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
xmatch_gaia.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) #get the closest match for each desi object
new_cols = ['GAIA_'+x  if 'desi' not in x  else x  for x in xmatch_gaia.columns]
xmatch_gaia.columns = new_cols
print(len(xmatch_gaia), 'gaia sources the closest filter')
xmatch_gaia.sample(5)

2981 gaia sources after cross-match
2956 gaia sources the closest filter


,GAIA_angDist,nway_desi_id,desi_ra,desi_dec,GAIA_DR3Name,GAIA_RAdeg,GAIA_DEdeg,GAIA_errHalfMaj,GAIA_errHalfMin,GAIA_errPosAng,GAIA_SolID,GAIA_Source,GAIA_RandomI,GAIA_e_RAdeg,GAIA_e_DEdeg,GAIA_Plx,GAIA_e_Plx,GAIA_RPlx,GAIA_PM,GAIA_pmRA,GAIA_e_pmRA,GAIA_pmDE,GAIA_e_pmDE,GAIA_RADEcor,GAIA_RAPlxcor,GAIA_RApmRAcor,GAIA_RApmDEcor,GAIA_DEPlxcor,GAIA_DEpmRAcor,GAIA_DEpmDEcor,GAIA_PlxpmRAcor,GAIA_PlxpmDEcor,GAIA_pmRApmDEcor,GAIA_NAL,GAIA_NAC,GAIA_NgAL,GAIA_NbAL,GAIA_gofAL,GAIA_chi2AL,GAIA_epsi,GAIA_sepsi,GAIA_Solved,GAIA_APF,GAIA_nueff,GAIA_pscol,GAIA_e_pscol,GAIA_RApscolCorr,GAIA_DEpscolCorr,GAIA_PlxpscolCorr,GAIA_pmRApscolCorr,GAIA_pmDEpscolCorr,GAIA_MatchObsA,GAIA_Nper,GAIA_amax,GAIA_MatchObs,GAIA_IPDgofha,GAIA_IPDgofhp,GAIA_IPDfmp,GAIA_IPDfow,GAIA_RUWE,GAIA_Dup,GAIA_o_Gmag,GAIA_FG,GAIA_e_FG,GAIA_RFG,GAIA_Gmag,GAIA_e_Gmag,GAIA_o_BPmag,GAIA_FBP,GAIA_e_FBP,GAIA_RFBP,GAIA_BPmag,GAIA_e_BPmag,GAIA_o_RPmag,GAIA_FRP,GAIA_e_FRP,GAIA_RFRP,GAIA_RPmag,GAIA_e_RPmag,GAIA_E(BP/RP),GAIA_NBPcont,GAIA_NBPblend,GAIA_NRPcont,GAIA_NRPblend,GAIA_Mode,GAIA_BP-RP,GAIA_BP-G,GAIA_G-RP,GAIA_RV,GAIA_e_RV,GAIA_n_RV,GAIA_o_RV,GAIA_o_RVd,GAIA_RVNper,GAIA_RVS/N,GAIA_RVgof,GAIA_RVchi2,GAIA_RVTdur,GAIA_RVamp,GAIA_RVtempTeff,GAIA_RVtemplogg,GAIA_RVtemp[Fe/H],GAIA_Vatmparam,GAIA_vbroad,GAIA_e_Vbroad,GAIA_o_Vbroad,GAIA_GRVSmag,GAIA_e_GRVSmag,GAIA_o_GRVSmag,GAIA_RVSS/N,GAIA_VarFlag,GAIA_PQSO,GAIA_PGal,GAIA_PSS,GAIA_Teff,GAIA_b_Teff,GAIA_B_Teff,GAIA_logg,GAIA_b_logg,GAIA_B_logg,GAIA_[Fe/H],GAIA_b_[Fe/H],GAIA_B_[Fe/H],GAIA_Dist,GAIA_b_Dist,GAIA_B_Dist,GAIA_A0,GAIA_b_A0,GAIA_B_A0,GAIA_AG,GAIA_b_AG,GAIA_B_AG,GAIA_E(BP-RP),GAIA_b_E(BP-RP),GAIA_B_E(BP-RP),GAIA_Lib,GAIA_RAJ2000,GAIA_DEJ2000,GAIA_e_RAJ2000,GAIA_e_DEJ2000,GAIA_RADEcorJ2000
816,0.022160,9011_614511_3328,159.211417,59.015201,Gaia DR3 855500104600477184,159.211414,59.015207,0.012,0.004,30,1636148068921376768,855500104600477184,821353531,6.9364,10.7005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87,0,86,1,12.2647,357.06,21.332,12.61800,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,6,45.234016,11,0.294300,71.11,0,0,NaN,0,86,45.05846,1.269700,35.4874,21.552925,0.030719,5,52.6991,9.91195,5.31672,21.034035,0.204231,9,127.4990,10.13570,12.5792,19.484125,0.086395,3.999,0.0,0.0,0.0,0.0,0,1.549910,-0.518890,2.068800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_AVAILABLE,0.000067,0.000012,0.999628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,159.211414,59.015207,6.936369,10.700462,0.7897
1,0.000409,9011_606779_284,154.769507,56.559155,Gaia DR3 853371518808768256,154.769507,56.559155,0.001,0.001,148,1636148068921376768,853371518808768256,408285686,0.5501,0.5782,-1.2416,0.7789,-1.5939,0.626,0.611,0.930,-0.134,0.669274,-0.0998,0.0740,-0.0490,0.0745,-0.4995,0.1100,-0.3539,-0.0055,0.0706,-0.2073,284,0,283,1,0.3360,313.27,1.493,0.71489,31,0,1.548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,19,1.317837,36,0.006528,43.03,0,0,1.013,0,289,111.68520,0.816078,136.8560,20.567377,0.008398,25,63.1170,7.90376,7.98569,20.838177,0.135989,25,73.3608,6.70168,10.9466,20.084236,0.099257,1.222,0.0,0.0,0.0,1.0,0,0.753941,0.270800,0.483141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_AVAILABLE,0.941188,0.000004,0.058808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154.769502,56.559156,14.918794,10.926400,-0.2114
531,0.000461,9011_611484_3453,158.359113,58.022455,Gaia DR3 855021645247596160,158.359113,58.022456,0.001,0.001,24,1636148068921376768,855021645247596160,234443374,0.6786,1.0142,-0.0344,1.2764,-0.0270,0.552,-0.541,0.858,0.110,1.131717,0.4573,-0.1660,-0.4180,-0.1564,-0.4081,-0.2859,-0.2855,0.2896,0.0086,0.5818,241,0,241,0,-0.3293,244.34,0.000,0.00000,95,0,NaN,1.0072,0.2120,0.048,-0.164,0.130,0.369,0.272,28,14,1.829057,29,0.082570,54.96,0,0,0.983,0,236,88.17763,1.153990,76.4111,20.823971,0.014474,26,96.4249,8.73815,11.03490,20.378069,0.098430,23,83.5675,

In [5]:
xmatch_gaia.GAIA_angDist.describe()

count    2956.000000
mean        0.008634
std         0.021257
min         0.000008
25%         0.000320
50%         0.001032
75%         0.006211
max         0.239019
Name: GAIA_angDist, dtype: float64

In [6]:
desi_gaia = desi.merge(xmatch_gaia, on=['nway_desi_id', 'desi_ra', 'desi_dec'], how='right')

## with SDSS DR16

https://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=V/154

In [7]:
xmatch_sdss = pd.read_csv(data_path+'desi_LH_coords_sdssdr16.csv')
print(len(xmatch_sdss), 'sdss sources after cross-match')
xmatch_sdss = xmatch_sdss.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
xmatch_sdss = xmatch_sdss.query('~spCl.isna()') #remove objects without classification
print(len(xmatch_sdss), 'sdss sources the classification')
xmatch_sdss.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) # get the closest match for each desi object
new_cols = ['SDSS_'+x  if 'desi' not in x  else x  for x in xmatch_sdss.columns]
xmatch_sdss.columns = new_cols
print(len(xmatch_sdss), 'sdss sources the closest filter')
xmatch_sdss

18737 sdss sources after cross-match
2434 sdss sources the classification
2418 sdss sources the closest filter


,SDSS_angDist,nway_desi_id,desi_ra,desi_dec,SDSS_objID,SDSS_RA_ICRS,SDSS_DE_ICRS,SDSS_errHalfMaj,SDSS_errHalfMin,SDSS_errPosAng,SDSS_mode,SDSS_class,SDSS_clean,SDSS_e_RA_ICRS,SDSS_e_DE_ICRS,SDSS_umag,SDSS_gmag,SDSS_rmag,SDSS_imag,SDSS_zmag,SDSS_e_umag,SDSS_e_gmag,SDSS_e_rmag,SDSS_e_imag,SDSS_e_zmag,SDSS_zsp,SDSS_e_zsp,SDSS_f_zsp,SDSS_spCl,SDSS_subCl,SDSS_pmRA,SDSS_pmDE,SDSS_e_pmRA,SDSS_e_pmDE,SDSS_sigRA,SDSS_sigDE,SDSS_zph,SDSS_e_zph,SDSS_<zph>,SDSS_Q,SDSS_SDSS16,SDSS_Sp-ID,SDSS_MJD
11218,0.044010,9011_598549_854,161.461856,54.123404,1237657769639215311,161.461870,54.123395,0.009,0.009,90,1,6,1,0.009,0.009,20.246,20.042,19.707,19.422,19.360,0.048,0.019,0.017,0.019,0.068,1.50396,0.00034,0.0,QSO,BROADLINE,2.73,-2.14,4.05,4.05,551.72,359.62,NaN,NaN,NaN,3,SDSS J104550.84+540724.2,8173-57375-0981,52283.3136
9599,0.057747,9011_599395_499,160.853456,54.274962,1237657220412145758,160.853442,54.274976,0.006,0.006,90,1,6,1,0.006,0.006,19.061,18.911,18.796,18.958,18.987,0.022,0.009,0.010,0.014,0.041,0.97346,0.00020,0.0,QSO,BROADLINE,-0.40,3.69,3.27,3.27,123.74,160.18,NaN,NaN,NaN,3,SDSS J104324.82+541629.9,906-52368-0132,52233.4896
11156,0.039859,9011_599396_2176,161.434078,54.186100,1237657769639215423,161.434088,54.186090,0.016,0.015,90,1,6,1,0.016,0.015,20.942,20.626,20.501,20.456,20.159,0.078,0.028,0.029,0.041,0.126,0.77491,0.00021,0.0,QSO,BROADLINE,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,3,SDSS J104544.18+541109.9,8173-57375-0968,52283.3136
11259,0.062339,9011_599396_2647,161.485139,54.201269,1237657769639215439,161.485150,54.201253,0.022,0.022,0,1,6,1,0.022,0.022,21.109,20.977,20.796,20.603,20.659,0.091,0.037,0.037,0.047,0.198,2.02040,0.00065,0.0,QSO,NaN,4.76,22.89,3.98,3.98,183.09,966.04,NaN,NaN,NaN,3,SDSS J104556.43+541204.5,8173-57375-0982,52283.3136
11484,0.045899,9011_599396_3413,161.572180,54.348553,1237657220412342517,161.572202,54.348552,0.018,0.017,90,1,6,1,0.018,0.017,20.714,20.464,20.224,20.062,19.923,0.070,0.022,0.028,0.030,0.091,1.56729,0.00049,0.0,QSO,BROADLINE,4.49,1.37,5.73,5.73,177.99,137.70,NaN,NaN,NaN,3,SDSS J104617.32+542054.7,8172-57423-0496,52233.4908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9493,0.023750,9011_620307_2187,160.809061,60.958682,1237653618545066138,160.809054,60.958677,0.008,0.008,90,1,6,1,0.008,0.008,19.718,19.620,19.204,19.151,19.190,0.033,0.014,0.013,0.019,0.065,1.19342,0.00021,0.0,QSO,BROADLINE,2.86,-3.62,3.44,3.44,173.41,196.74,NaN,NaN,NaN,3,SDSS J104314.17+605731.2,7096-56683-0660,51875.4578
8945,0.033750,9011_620307_596,160.596115,61.117255,1237655370893688838,160.596104,61.117247,0.003,0.003,90,1,6,1,0.003,0.003,18.343,18.086,17.878,17.909,17.913,0.014,0.006,0.006,0.008,0.021,0.95777,0.00024,0.0,QSO,BROADLINE,1.73,-0.67,2.95,2.95,140.93,133.36,NaN,NaN,NaN,3,SDSS J104223.06+610702.0,561-52295-0376,52047.1623
10536,0.115783,9011_620308_1164,161.198300,61.025895,1237653618545131614,161.198234,61.025889,0.015,0.015,90,1,6,1,0.015,0.015,20.639,20.070,19.808,19.696,19.666,0.062,0.018,0.020,0.027,0.101,0.78407,0.00004,0.0,QSO,BROADLINE,-1.53,2.06,3.72,3.72,215.13,93.35,NaN,NaN,NaN,3,SDSS J104447.57+610133.2,7096-56683-0700,51875.4582
7180,0.091026,9011_621005_665,159.919707,61.235864,1237653619081805917,159.919684,61.235842,0.017,0.017,90,1,6,1,0.017,0.017,20.383,20.058,20.004,20.002,19.571,0.059,0.018,0.023,0.035,0.094,0.79690,0.00014,0.0,QSO,NaN,0.30,-1.49,3.98,3.98,208.10,192.48,NaN,NaN,NaN,3,SDSS J103940.72+611409.0,561-52295-0330,51875.4569


In [8]:
xmatch_sdss.SDSS_spCl.value_counts()

QSO       2114
GALAXY     300
STAR         4
Name: SDSS_spCl, dtype: int64

In [8]:
xmatch_sdss.groupby(['SDSS_spCl']).SDSS_subCl.value_counts()

SDSS_spCl  SDSS_subCl 
GALAXY     STARFORMING      32
           AGN              16
           STARBURST        12
           BROADLINE         9
QSO        BROADLINE      1621
           STARBURST        60
           AGN               6
           STARFORMING       3
STAR       O9.5Iae           2
           F9                1
           O8e               1
Name: SDSS_subCl, dtype: int64

In [9]:
desi_sdss = desi.merge(xmatch_sdss, on=['nway_desi_id', 'desi_ra', 'desi_dec'], how='right')


## with SIMBAD

In [10]:
# xmatch_simbad = pd.read_csv(data_path+'desi_LH_coords_simbad.csv')
# xmatch_simbad = xmatch_simbad.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
# #xmatch_simbad.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) # get the closest match for each desi object
# new_cols = ['XMATCH_Simbad_'+x  if 'desi' not in x  else x  for x in xmatch_simbad.columns]
# xmatch_simbad.columns = new_cols
# xmatch_simbad

## with millquas

https://heasarc.gsfc.nasa.gov/W3Browse/all/milliquas.html


https://quasars.org/Milliquas-ReadMe.txt

In [11]:
# f = astropy.io.fits.open(data_path+'milliquas.fits')
# mask_ra = (f[1].data['RA'] > desi[['RA_fin', 'DEC_fin']].min().RA_fin) & (f[1].data['RA'] < desi[['RA_fin', 'DEC_fin']].max().RA_fin)
# mask_dec = (f[1].data['DEC'] > desi[['RA_fin', 'DEC_fin']].min().DEC_fin) & (f[1].data['DEC'] < desi[['RA_fin', 'DEC_fin']].max().DEC_fin)
# mask = mask_ra & mask_dec
# data_tmp = f[1].data[mask]
# milq = pd.DataFrame(data_tmp)
# milq.to_csv(data_path+'milliquas_LH.csv', index=False)
#milq = pd.read_csv(data_path+'milliquas_LH.csv')
xmatch_milq = pd.read_csv(data_path+'desi_LH_coords_milliquas.csv')
xmatch_milq = xmatch_milq.sort_values(by=['nway_desi_id', 'angDist'], ascending=True)
print('milq LH match (2arcsec)', len(xmatch_milq))

xmatch_milq.drop_duplicates(subset = ['nway_desi_id'], keep='first', inplace=True) # get the closest match for each desi object
new_cols = ['MILQ_'+x  if 'desi' not in x  else x  for x in xmatch_milq.columns]
xmatch_milq.columns = new_cols
print('milq LH match (2arcsec) - closest', len(xmatch_milq))

xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.str.replace('R', '')
xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.str.replace('X', '')
xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.str.replace('2', '')
type_dict_milq = {
    'Q': 'QSO',
    'A': 'AGN',
    'B': 'BLAZAR',
    'L': 'QSO-LENSED',
    'K': 'QSO-NL',
    'N': 'AGN-NL',
    'G': 'GALAXY',
    'S': 'STAR',
    'C': 'CV',
    'M': 'MOVING-OBJECT',
    '': 'RADIO-SOURCE',
}
xmatch_milq['MILQ_TYPE'] = xmatch_milq.MILQ_TYPE.map(type_dict_milq)
desi_milq = desi.merge(xmatch_milq, on=['nway_desi_id', 'desi_ra', 'desi_dec'], how='right')

xmatch_milq



milq LH match (2arcsec) 2811
milq LH match (2arcsec) - closest 2793


,MILQ_angDist,nway_desi_id,desi_ra,desi_dec,MILQ_RA,MILQ_DEC,MILQ_NAME,MILQ_TYPE,MILQ_RMAG,MILQ_BMAG,MILQ_COMMENT,MILQ_R,MILQ_B,MILQ_Z,MILQ_CITE,MILQ_ZCITE,MILQ_RXPCT,MILQ_QPCT,MILQ_XNAME,MILQ_RNAME,MILQ_LOBE1,MILQ_LOBE2
1167,0.009984,9011_599395_499,160.853456,54.274962,160.853453,54.274964,SDSS J104324.82+541629.9,QSO,18.80,18.93,gG,-,-,0.973,DR16Q,DR16Q,NaN,NaN,NaN,NaN,NaN,NaN
1410,0.026362,9011_599396_2176,161.434078,54.186100,161.434084,54.186107,SDSS J104544.18+541109.9,QSO,20.52,20.62,gmG,1,-,0.775,DR16Q,DR16,79.0,70.0,2RXP J104543.9+541107,NaN,NaN,NaN
1427,0.103350,9011_599396_2647,161.485139,54.201269,161.485112,54.201292,SDSS J104556.43+541204.5,QSO,20.84,21.00,gG,1,-,2.020,DR16Q,DR16,NaN,NaN,NaN,NaN,NaN,NaN
1469,0.007490,9011_599396_3413,161.572180,54.348553,161.572178,54.348552,SDSS J104617.32+542054.7,QSO,20.24,20.48,gG,1,1,1.568,DR16Q,DR16,99.0,78.0,1WGA J1046.2+5420,VLAJ104617.28+542054.7,FIRST J104617.2+542054,NVSS J104617.6+542058
1581,0.009694,9011_599397_1494,161.827714,54.321634,161.827716,54.321636,1WGA J1047.2+5419,QSO,18.24,18.32,gmG,-,-,1.834,1218,DR16Q,100.0,98.0,2RXP J104718.5+541919,VLAJ104718.61+541917.9,NVSS J104718.6+541917,FIRST J104718.6+541917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,0.001890,9011_620307_2187,160.809061,60.958682,160.809061,60.958682,SDSS J104314.17+605731.2,QSO,19.23,19.64,gmG,-,-,1.193,DR16Q,DR16Q,NaN,NaN,NaN,NaN,NaN,NaN
1077,0.002527,9011_620307_596,160.596115,61.117255,160.596114,61.117254,SDSS J104223.06+610702.0,QSO,17.90,18.11,gG,-,-,0.958,DR16Q,DR16Q,NaN,NaN,NaN,NaN,NaN,NaN
1300,0.032367,9011_620308_1164,161.198300,61.025895,161.198310,61.025903,SDSS J104447.57+610133.2,QSO,19.83,20.09,gmG,-,-,0.784,DR16Q,DR16Q,NaN,NaN,NaN,NaN,NaN,NaN
841,0.018016,9011_621005_665,159.919707,61.235864,159.919700,61.235860,SDSS J103940.72+611409.0,QSO,20.05,20.08,gG,-,-,0.797,DR16Q,DR16Q,NaN,NaN,NaN,NaN,NaN,NaN


### compare SDSS and MILQ

In [12]:
xmatch_sdss_milq = desi_sdss.merge(xmatch_milq, on=['nway_desi_id', 'desi_ra', 'desi_dec'], how='right')
test_cols = ['SDSS_angDist', 'MILQ_angDist','SDSS_SDSS16', 'SDSS_spCl', 'SDSS_subCl',  'MILQ_NAME', 'MILQ_TYPE', 'SDSS_zsp', 'MILQ_Z']

In [13]:
tmp_df = xmatch_sdss_milq.query("~SDSS_SDSS16.isna() & ~MILQ_NAME.isna() & (MILQ_NAME == SDSS_SDSS16)  ")[test_cols]
print('sdss -  milq complete match', len(tmp_df))
tmp_df

sdss -  milq complete match 1845


,SDSS_angDist,MILQ_angDist,SDSS_SDSS16,SDSS_spCl,SDSS_subCl,MILQ_NAME,MILQ_TYPE,SDSS_zsp,MILQ_Z
0,0.057747,0.009984,SDSS J104324.82+541629.9,QSO,BROADLINE,SDSS J104324.82+541629.9,QSO,0.97346,0.973
1,0.039859,0.026362,SDSS J104544.18+541109.9,QSO,BROADLINE,SDSS J104544.18+541109.9,QSO,0.77491,0.775
2,0.062339,0.103350,SDSS J104556.43+541204.5,QSO,NaN,SDSS J104556.43+541204.5,QSO,2.02040,2.020
3,0.045899,0.007490,SDSS J104617.32+542054.7,QSO,BROADLINE,SDSS J104617.32+542054.7,QSO,1.56729,1.568
6,0.025707,0.016047,SDSS J104139.89+543318.4,QSO,BROADLINE,SDSS J104139.89+543318.4,QSO,2.08642,2.086
...,...,...,...,...,...,...,...,...,...
2804,0.023750,0.001890,SDSS J104314.17+605731.2,QSO,BROADLINE,SDSS J104314.17+605731.2,QSO,1.19342,1.193
2805,0.033750,0.002527,SDSS J104223.06+610702.0,QSO,BROADLINE,SDSS J104223.06+610702.0,QSO,0.95777,0.958
2806,0.115783,0.032367,SDSS J104447.57+610133.2,QSO,BROADLINE,SDSS J104447.57+610133.2,QSO,0.78407,0.784
2807,0.091026,0.018016,SDSS J103940.72+611409.0,QSO,NaN,SDSS J103940.72+611409.0,QSO,0.79690,0.797


In [14]:
tmp_df = xmatch_sdss_milq.query("~SDSS_SDSS16.isna() & ~MILQ_NAME.isna() & (MILQ_NAME != SDSS_SDSS16)  ")[test_cols]
tmp_df['z_diff'] = np.abs(tmp_df['MILQ_Z'] - tmp_df['SDSS_zsp'])
tmp_df = tmp_df.sort_values(by=['z_diff'], ascending=False)
#max_z_diff = (tmp_df.MILQ_Z - tmp_df.SDSS_zsp).max()
#print('max z diff', max_z_diff)
print('sdss -  milq un match -', len(tmp_df))
print('sdss -  milq un match - max z diff', tmp_df.z_diff.max())
print('sdss -  milq un match - z diff < 0.02', np.sum(tmp_df.z_diff<0.02))

#print(tmp_df.z_diff.describe())
tmp_df.head(15)

sdss -  milq un match - 389
sdss -  milq un match - max z diff 1.8691799999999998
sdss -  milq un match - z diff < 0.02 378


,SDSS_angDist,MILQ_angDist,SDSS_SDSS16,SDSS_spCl,SDSS_subCl,MILQ_NAME,MILQ_TYPE,SDSS_zsp,MILQ_Z,z_diff
2237,0.011496,0.018346,SDSS J104255.79+590427.6,GALAXY,NaN,CLANS J10429+5904,AGN,0.89082,2.760,1.86918
2076,0.090863,0.468537,SDSS J104604.05+584424.6,GALAXY,NaN,CLANS J10460+5844,QSO,0.00000,1.810,1.81000
2395,0.155465,0.156834,SDSS J104539.63+591226.7,GALAXY,NaN,CLANS J10456+5912,QSO,0.00000,1.760,1.76000
116,0.073453,0.010610,SDSS J105030.05+550542.1,GALAXY,NaN,LAMOSTJ105030.06+550542.2,QSO,0.00000,1.451,1.45100
2272,0.030755,0.017518,SDSS J104817.99+585954.1,GALAXY,NaN,J104817.99+585954.0,MOVING-OBJECT,0.00000,0.900,0.90000
1024,0.063037,0.001082,SDSS J103744.30+571155.6,STAR,O8e,RXS J10377+5711,BLAZAR,-0.00340,0.830,0.83340
1025,0.063037,0.001082,SDSS J103744.30+571155.6,STAR,O8e,RXS J10377+5711,BLAZAR,-0.00340,0.830,0.83340
1042,0.130996,0.042082,SDSS J104227.46+571200.0,GALAXY,NaN,SDSS J104227.46+571159.9,QSO,0.00000,0.826,0.82600
2397,0.142743,0.142744,SDSS J104422.64+591304.1,QSO,BROADLINE,CLANS J10443+5913,QSO,0.48957,0.953,0.46343
75,0.073017,0.042742,SDSS J104745.82+543741.0,GALAXY,NaN,1ES 1044+549,BLAZAR,0.62154,0.540,0.08154


In [15]:
xmatch_sdss_milq.query("SDSS_SDSS16.isna() & ~MILQ_NAME.isna() & (MILQ_NAME != SDSS_SDSS16)  ")[test_cols]

,SDSS_angDist,MILQ_angDist,SDSS_SDSS16,SDSS_spCl,SDSS_subCl,MILQ_NAME,MILQ_TYPE,SDSS_zsp,MILQ_Z
5,NaN,0.000114,NaN,NaN,NaN,J104828.83+541713.3,RADIO-SOURCE,NaN,0.600
25,NaN,0.007243,NaN,NaN,NaN,J104708.54+542334.9,RADIO-SOURCE,NaN,1.600
33,NaN,0.000657,NaN,NaN,NaN,Gaia 850504645318467712,QSO,NaN,0.500
61,NaN,0.026344,NaN,NaN,NaN,J104645.06+543931.2,RADIO-SOURCE,NaN,0.100
65,NaN,0.032922,NaN,NaN,NaN,J104705.11+544443.3,RADIO-SOURCE,NaN,0.100
...,...,...,...,...,...,...,...,...,...
2766,NaN,0.129595,NaN,NaN,NaN,J104341.74+603005.5,RADIO-SOURCE,NaN,NaN
2769,NaN,0.007351,NaN,NaN,NaN,SDSS J104528.97+602536.1,QSO,NaN,1.292
2775,NaN,0.161575,NaN,NaN,NaN,J104914.41+603141.9,RADIO-SOURCE,NaN,NaN
2782,NaN,0.029706,NaN,NaN,NaN,F2M J103841.70+604402.1,QSO,NaN,0.515


In [16]:
xmatch_sdss_milq.query("~SDSS_SDSS16.isna() & SDSS_spCl=='GALAXY'")[test_cols]

,SDSS_angDist,MILQ_angDist,SDSS_SDSS16,SDSS_spCl,SDSS_subCl,MILQ_NAME,MILQ_TYPE,SDSS_zsp,MILQ_Z
28,0.093418,0.018796,SDSS J104902.05+542348.8,GALAXY,NaN,PGC 2468439,GALAXY,0.24914,0.249
39,0.024300,0.000048,SDSS J104110.90+544953.2,GALAXY,STARFORMING,SDSS J104110.90+544953.2,AGN,0.15793,0.158
43,0.069605,0.178189,SDSS J104151.56+544547.8,GALAXY,AGN,PGC 2479529,AGN-NL,0.08700,0.087
54,0.053368,0.081601,SDSS J104407.53+544626.9,GALAXY,NaN,PGC 2479867,GALAXY,0.12718,0.127
60,0.019583,0.036828,SDSS J104628.80+544944.3,GALAXY,NaN,PGC 2481647,BLAZAR,0.24921,0.249
...,...,...,...,...,...,...,...,...,...
2665,0.081949,0.077360,SDSS J105747.67+594713.2,GALAXY,STARFORMING,SDSS J105747.68+594713.3,AGN,0.15946,0.159
2700,0.042889,0.037975,SDSS J104755.14+595527.6,GALAXY,NaN,PGC 2595862,GALAXY,0.11507,0.115
2701,0.042889,0.037975,SDSS J104755.14+595527.6,GALAXY,NaN,PGC 2595862,GALAXY,0.11507,0.115
2763,0.064441,0.049818,SDSS J104403.31+603133.2,GALAXY,NaN,PGC 2603347,GALAXY,0.11631,0.116


In [17]:
xmatch_sdss_milq.query("~SDSS_SDSS16.isna() & MILQ_NAME.isna() & (MILQ_NAME != SDSS_SDSS16)  ")

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2w3w4,nway_nnmag_grzw1w2,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_duplicated,nway_is_confused,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,SDSS_angDist,SDSS_objID,SDSS_RA_ICRS,SDSS_DE_ICRS,SDSS_errHalfMaj,SDSS_errHalfMin,SDSS_errPosAng,SDSS_mode,SDSS_class,SDSS_clean,SDSS_e_RA_ICRS,SDSS_e_DE_ICRS,SDSS_umag,SDSS_gmag,SDSS_rmag,SDSS_imag,SDSS_zmag,SDSS_e_umag,SDSS_e_gmag,SDSS_e_rmag,SDSS_e_imag,SDSS_e_zmag,SDSS_zsp,SDSS_e_zsp,SDSS_f_zsp,SDSS_spCl,SDSS_subCl,SDSS_pmRA,SDSS_pmDE,SDSS_e_pmRA,SDSS_e_pmDE,SDSS_sigRA,SDSS_sigDE,SDSS_zph,SDSS_e_zph,SDSS_<zph>,SDSS_Q,SDSS_SDSS16,SDSS_Sp-ID,SDSS_MJD,MILQ_angDist,MILQ_RA,MILQ_DEC,MILQ_NAME,MILQ_TYPE,MILQ_RMAG,MILQ_BMAG,MILQ_COMMENT,MILQ_R,MILQ_B,MILQ_Z,MILQ_CITE,MILQ_ZCITE,MILQ_RXPCT,MILQ_QPCT,MILQ_XNAME,MILQ_RNAME,MILQ_LOBE1,MILQ_LOBE2


In [18]:
xmatch_sdss_milq.groupby('MILQ_TYPE')['SDSS_spCl', 'SDSS_subCl'].value_counts()

/var/folders/hk/rctb6p8n16nbrbnjrwzrc8680000gp/T/ipykernel_89135/2094432551.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  xmatch_sdss_milq.groupby('MILQ_TYPE')['SDSS_spCl', 'SDSS_subCl'].value_counts()


MILQ_TYPE  SDSS_spCl  SDSS_subCl 
AGN        QSO        STARBURST        52
                      BROADLINE        15
           GALAXY     STARFORMING      12
                      AGN               7
           QSO        AGN               4
                      STARFORMING       2
           GALAXY     BROADLINE         1
AGN-NL     GALAXY     AGN              10
           QSO        STARBURST        10
           GALAXY     STARFORMING       3
           QSO        STARFORMING       1
           GALAXY     STARBURST         1
                      BROADLINE         1
           QSO        AGN               1
BLAZAR     STAR       O8e               2
GALAXY     GALAXY     STARFORMING       5
                      STARBURST         4
                      BROADLINE         4
QSO        QSO        BROADLINE      1607
                      AGN               1
                      STARBURST         1
QSO-NL     QSO        BROADLINE         1
STAR       STAR       O9.5Iae           2


In [19]:
xmatch_sdss_milq.groupby(['SDSS_spCl', 'SDSS_subCl'])['MILQ_TYPE'].value_counts()

SDSS_spCl  SDSS_subCl   MILQ_TYPE
GALAXY     AGN          AGN-NL         10
                        AGN             7
           BROADLINE    GALAXY          4
                        AGN             1
                        AGN-NL          1
           STARBURST    GALAXY          4
                        AGN-NL          1
           STARFORMING  AGN            12
                        GALAXY          5
                        AGN-NL          3
QSO        AGN          AGN             4
                        AGN-NL          1
                        QSO             1
           BROADLINE    QSO          1607
                        AGN            15
                        QSO-NL          1
           STARBURST    AGN            52
                        AGN-NL         10
                        QSO             1
           STARFORMING  AGN             2
                        AGN-NL          1
STAR       O8e          BLAZAR          2
           O9.5Iae      STAR            2


# Join all matches

In [20]:
#merge desi_gaia with desi_sdss with desi to create a new desi_gaia_sdss catalog 
#https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
from functools import reduce
data_frames = [desi, desi_gaia, desi_sdss, desi_milq]

desi_matched = reduce(lambda  left,right: pd.merge(left,right,on=list(desi.columns.values),
                                            how='outer'), data_frames).fillna(np.nan)

desi_matched.fillna(np.nan, inplace=True)

desi_matched

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2w3w4,nway_nnmag_grzw1w2,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_duplicated,nway_is_confused,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,GAIA_angDist,GAIA_DR3Name,GAIA_RAdeg,GAIA_DEdeg,GAIA_errHalfMaj,GAIA_errHalfMin,GAIA_errPosAng,GAIA_SolID,GAIA_Source,GAIA_RandomI,GAIA_e_RAdeg,GAIA_e_DEdeg,GAIA_Plx,GAIA_e_Plx,GAIA_RPlx,GAIA_PM,GAIA_pmRA,GAIA_e_pmRA,GAIA_pmDE,GAIA_e_pmDE,GAIA_RADEcor,GAIA_RAPlxcor,GAIA_RApmRAcor,GAIA_RApmDEcor,GAIA_DEPlxcor,GAIA_DEpmRAcor,GAIA_DEpmDEcor,GAIA_PlxpmRAcor,GAIA_PlxpmDEcor,GAIA_pmRApmDEcor,GAIA_NAL,GAIA_NAC,GAIA_NgAL,GAIA_NbAL,GAIA_gofAL,GAIA_chi2AL,GAIA_epsi,GAIA_sepsi,GAIA_Solved,GAIA_APF,GAIA_nueff,GAIA_pscol,GAIA_e_pscol,GAIA_RApscolCorr,GAIA_DEpscolCorr,GAIA_PlxpscolCorr,GAIA_pmRApscolCorr,GAIA_pmDEpscolCorr,GAIA_MatchObsA,GAIA_Nper,GAIA_amax,GAIA_MatchObs,GAIA_IPDgofha,GAIA_IPDgofhp,GAIA_IPDfmp,GAIA_IPDfow,GAIA_RUWE,GAIA_Dup,GAIA_o_Gmag,GAIA_FG,GAIA_e_FG,GAIA_RFG,GAIA_Gmag,GAIA_e_Gmag,GAIA_o_BPmag,GAIA_FBP,GAIA_e_FBP,GAIA_RFBP,GAIA_BPmag,GAIA_e_BPmag,GAIA_o_RPmag,GAIA_FRP,GAIA_e_FRP,GAIA_RFRP,GAIA_RPmag,GAIA_e_RPmag,GAIA_E(BP/RP),GAIA_NBPcont,GAIA_NBPblend,GAIA_NRPcont,GAIA_NRPblend,GAIA_Mode,GAIA_BP-RP,GAIA_BP-G,GAIA_G-RP,GAIA_RV,GAIA_e_RV,GAIA_n_RV,GAIA_o_RV,GAIA_o_RVd,GAIA_RVNper,GAIA_RVS/N,GAIA_RVgof,GAIA_RVchi2,GAIA_RVTdur,GAIA_RVamp,GAIA_RVtempTeff,GAIA_RVtemplogg,GAIA_RVtemp[Fe/H],GAIA_Vatmparam,GAIA_vbroad,GAIA_e_Vbroad,GAIA_o_Vbroad,GAIA_GRVSmag,GAIA_e_GRVSmag,GAIA_o_GRVSmag,GAIA_RVSS/N,GAIA_VarFlag,GAIA_PQSO,GAIA_PGal,GAIA_PSS,GAIA_Teff,GAIA_b_Teff,GAIA_B_Teff,GAIA_logg,GAIA_b_logg,GAIA_B_logg,GAIA_[Fe/H],GAIA_b_[Fe/H],GAIA_B_[Fe/H],GAIA_Dist,GAIA_b_Dist,GAIA_B_Dist,GAIA_A0,GAIA_b_A0,GAIA_B_A0,GAIA_AG,GAIA_b_AG,GAIA_B_AG,GAIA_E(BP-RP),GAIA_b_E(BP-RP),GAIA_B_E(BP-RP),GAIA_Lib,GAIA_RAJ2000,GAIA_DEJ2000,GAIA_e_RAJ2000,GAIA_e_DEJ2000,GAIA_RADEcorJ2000,SDSS_angDist,SDSS_objID,SDSS_RA_ICRS,SDSS_DE_ICRS,SDSS_errHalfMaj,SDSS_errHalfMin,SDSS_errPosAng,SDSS_mode,SDSS_class,SDSS_clean,SDSS_e_RA_ICRS,SDSS_e_DE_ICRS,SDSS_umag,SDSS_gmag,SDSS_rmag,SDSS_imag,SDSS_zmag,SDSS_e_umag,SDSS_e_gmag,SDSS_e_rmag,SDSS_e_imag,SDSS_e_zmag,SDSS_zsp,SDSS_e_zsp,SDSS_f_zsp,SDSS_spCl,SDSS_subCl,SDSS_pmRA,SDSS_pmDE,SDSS_e_pmRA,SDSS_e_pmDE,SDSS_sigRA,SDSS_sigDE,SDSS_zph,SDSS_e_zph,SDSS_<zph>,SDSS_Q,SDSS_SDSS16,SDSS_Sp-ID,SDSS_MJD,MILQ_angDist,MILQ_RA,MILQ_DEC,MILQ_NAME,MILQ_TYPE,MILQ_RMAG,MILQ_BMAG,MILQ_COMMENT,MILQ_R,MILQ_B,MILQ_Z,MILQ_CITE,MILQ_ZCITE,MILQ_RXPCT,MILQ_QPCT,MILQ_XNAME,MILQ_RNAME,MILQ_LOBE1,MILQ_LOBE2
0,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,0.006185,7.077317,154.763934,56.505585,853324209744036352,7.576342,1,0,20.950485,-1.0,0,SDSS J101902.92+563016.4,0.949746,QSO,7.576898,1,0,0,2,16.915001,16.554001,12.382,12.1,5.0,0.7,2,1,1,1237658303814435106,9216733310671409152,7.544174,68469189470,4.058504,3.313348,5.001804,8.11

In [21]:
#[x for x in desi_gaia_sdss.columns if 'SDSS' in x]

## preliminary classification

GAIA

In [22]:

th = 0.9
qso_mask = desi_matched.eval('(GAIA_PQSO > @th) ')
desi_matched.loc[qso_mask, 'class_GAIA_class'] = 'QSO'

galaxy_mask = desi_matched.eval('(GAIA_PGal > @th) ')
desi_matched.loc[galaxy_mask, 'class_GAIA_class'] = 'GALAXY'



th = 3
star_mask = desi_matched.eval('(GAIA_RPlx > @th) | (GAIA_pmRA/GAIA_e_pmRA > @th) | (GAIA_pmDE/GAIA_e_pmDE > @th)')
desi_matched.loc[star_mask, 'class_GAIA_class'] = 'STAR'

#desi_matched.loc[desi_matched.class_GAIA_class.isna() & ~desi_matched.GAIA_DR3Name.isna(), 'class_GAIA_class'] = 'undef'


In [23]:
print('NOTE NANS ARE NOT PRINTER IN .value_counts()')
desi_matched.class_GAIA_class.value_counts()

NOTE NANS ARE NOT PRINTER IN .value_counts()


QSO       1502
STAR       479
GALAXY     244
Name: class_GAIA_class, dtype: int64

SDSS

In [24]:
sdss_class_mask = desi_matched.eval('~SDSS_spCl.isna() ')
desi_matched.loc[sdss_class_mask, 'class_SDSS_class'] = desi_matched.SDSS_spCl[sdss_class_mask]
assert len(desi_sdss.SDSS_spCl) == np.sum(sdss_class_mask), f'SDSS_spCl and class_SDSS_class have different lengths {len(desi_sdss.SDSS_spCl)} {len(desi_matched.class_SDSS_class)}'

In [25]:
desi_matched.class_SDSS_class.value_counts()

QSO       2125
GALAXY     304
STAR         5
Name: class_SDSS_class, dtype: int64

MILQ

In [26]:
mask_extragal = desi_matched.eval(" MILQ_TYPE in ['QSO', 'AGN', 'AGN-NL', 'QSO-NL', 'BLAZAR']") #GALAXY
desi_matched.loc[mask_extragal, 'class_MILQ_class'] = 'QSO'

mask_extragal = desi_matched.eval(" MILQ_TYPE in ['GALAXY']") #GALAXY
desi_matched.loc[mask_extragal, 'class_MILQ_class'] = 'GALAXY'


## Compare classes

In [27]:
print('both SDSS and MILQ have class and they are the same:')
print(len(desi_matched.query("~class_MILQ_class.isna() & ~class_SDSS_class.isna() & class_MILQ_class==class_SDSS_class")))
print('both SDSS and MILQ have class and they are different:')
print(len(desi_matched.query("~class_MILQ_class.isna() & ~class_SDSS_class.isna() & class_MILQ_class!=class_SDSS_class")))

print('SDSS class available only but not MILQ class:')
print(len(desi_matched.query("class_MILQ_class.isna() & ~class_SDSS_class.isna()")))

print('MILQ class available only but not SDSS class:')
print(len(desi_matched.query("class_SDSS_class.isna() & ~class_MILQ_class.isna()")))

both SDSS and MILQ have class and they are the same:
2175
both SDSS and MILQ have class and they are different:
50
SDSS class available only but not MILQ class:
209
MILQ class available only but not SDSS class:
206


In [28]:
desi_matched.groupby('class_GAIA_class').class_SDSS_class.value_counts()


class_GAIA_class  class_SDSS_class
GALAXY            GALAXY               140
                  QSO                   26
QSO               QSO                 1253
                  GALAXY                 8
                  STAR                   4
STAR              QSO                    9
                  GALAXY                 4
                  STAR                   1
Name: class_SDSS_class, dtype: int64

In [29]:
desi_matched.groupby('class_GAIA_class').class_MILQ_class.value_counts()

class_GAIA_class  class_MILQ_class
GALAXY            QSO                   55
                  GALAXY                41
QSO               QSO                 1276
                  GALAXY                 3
STAR              QSO                   12
                  GALAXY                 1
Name: class_MILQ_class, dtype: int64

When GAIA indicate star but SDSS says QSO

In [30]:
#[x for x in desi_gaia_sdss.columns if 'mag' in x]
mags_cols = ['desi_dered_mag_g', 'SDSS_gmag', 'GAIA_Gmag', 'desi_dered_mag_r', 'SDSS_rmag', 'desi_dered_mag_z', 'SDSS_zmag']
desi_matched.query('class_SDSS_class=="QSO" & class_GAIA_class=="STAR"')[mags_cols]


,desi_dered_mag_g,SDSS_gmag,GAIA_Gmag,desi_dered_mag_r,SDSS_rmag,desi_dered_mag_z,SDSS_zmag
1507,19.502940,19.916,19.483557,19.268133,19.699,18.157497,18.263
1677,20.445957,20.548,20.179214,20.267962,20.207,20.223413,20.076
3993,19.136795,19.438,19.713089,18.189823,18.344,17.804039,17.914
5318,20.943506,21.067,20.807379,20.660255,20.919,20.562060,20.572
7340,16.285994,16.335,18.401281,15.611326,15.646,15.086662,14.970
7341,16.285994,16.335,18.401281,15.611326,15.646,15.086662,14.970
8173,18.684580,18.615,20.412914,17.616146,17.603,16.813606,16.702
8446,18.769289,19.423,19.332888,18.124111,18.369,17.429146,17.493
8921,20.144867,20.079,20.459467,20.229116,20.161,19.908102,19.993


## Finalize classes

In [34]:
desi_matched

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,nway_desi_id,nway_Separation_EROSITA_DESI,nway_prob_has_match,nway_prob_this_match,nway_nnmag_grzw1w2w3w4,nway_nnmag_grzw1w2,nway_nnmag_grz,nway_n_match_flag_0,nway_n_match_flag_2,nway_is_duplicated,nway_is_confused,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,GAIA_angDist,GAIA_DR3Name,GAIA_RAdeg,GAIA_DEdeg,GAIA_errHalfMaj,GAIA_errHalfMin,GAIA_errPosAng,GAIA_SolID,GAIA_Source,GAIA_RandomI,GAIA_e_RAdeg,GAIA_e_DEdeg,GAIA_Plx,GAIA_e_Plx,GAIA_RPlx,GAIA_PM,GAIA_pmRA,GAIA_e_pmRA,GAIA_pmDE,GAIA_e_pmDE,GAIA_RADEcor,GAIA_RAPlxcor,GAIA_RApmRAcor,GAIA_RApmDEcor,GAIA_DEPlxcor,GAIA_DEpmRAcor,GAIA_DEpmDEcor,GAIA_PlxpmRAcor,GAIA_PlxpmDEcor,GAIA_pmRApmDEcor,GAIA_NAL,GAIA_NAC,GAIA_NgAL,GAIA_NbAL,GAIA_gofAL,GAIA_chi2AL,GAIA_epsi,GAIA_sepsi,GAIA_Solved,GAIA_APF,GAIA_nueff,GAIA_pscol,GAIA_e_pscol,GAIA_RApscolCorr,GAIA_DEpscolCorr,GAIA_PlxpscolCorr,GAIA_pmRApscolCorr,GAIA_pmDEpscolCorr,GAIA_MatchObsA,GAIA_Nper,GAIA_amax,GAIA_MatchObs,GAIA_IPDgofha,GAIA_IPDgofhp,GAIA_IPDfmp,GAIA_IPDfow,GAIA_RUWE,GAIA_Dup,GAIA_o_Gmag,GAIA_FG,GAIA_e_FG,GAIA_RFG,GAIA_Gmag,GAIA_e_Gmag,GAIA_o_BPmag,GAIA_FBP,GAIA_e_FBP,GAIA_RFBP,GAIA_BPmag,GAIA_e_BPmag,GAIA_o_RPmag,GAIA_FRP,GAIA_e_FRP,GAIA_RFRP,GAIA_RPmag,GAIA_e_RPmag,GAIA_E(BP/RP),GAIA_NBPcont,GAIA_NBPblend,GAIA_NRPcont,GAIA_NRPblend,GAIA_Mode,GAIA_BP-RP,GAIA_BP-G,GAIA_G-RP,GAIA_RV,GAIA_e_RV,GAIA_n_RV,GAIA_o_RV,GAIA_o_RVd,GAIA_RVNper,GAIA_RVS/N,GAIA_RVgof,GAIA_RVchi2,GAIA_RVTdur,GAIA_RVamp,GAIA_RVtempTeff,GAIA_RVtemplogg,GAIA_RVtemp[Fe/H],GAIA_Vatmparam,GAIA_vbroad,GAIA_e_Vbroad,GAIA_o_Vbroad,GAIA_GRVSmag,GAIA_e_GRVSmag,GAIA_o_GRVSmag,GAIA_RVSS/N,GAIA_VarFlag,GAIA_PQSO,GAIA_PGal,GAIA_PSS,GAIA_Teff,GAIA_b_Teff,GAIA_B_Teff,GAIA_logg,GAIA_b_logg,GAIA_B_logg,GAIA_[Fe/H],GAIA_b_[Fe/H],GAIA_B_[Fe/H],GAIA_Dist,GAIA_b_Dist,GAIA_B_Dist,GAIA_A0,GAIA_b_A0,GAIA_B_A0,GAIA_AG,GAIA_b_AG,GAIA_B_AG,GAIA_E(BP-RP),GAIA_b_E(BP-RP),GAIA_B_E(BP-RP),GAIA_Lib,GAIA_RAJ2000,GAIA_DEJ2000,GAIA_e_RAJ2000,GAIA_e_DEJ2000,GAIA_RADEcorJ2000,SDSS_angDist,SDSS_objID,SDSS_RA_ICRS,SDSS_DE_ICRS,SDSS_errHalfMaj,SDSS_errHalfMin,SDSS_errPosAng,SDSS_mode,SDSS_class,SDSS_clean,SDSS_e_RA_ICRS,SDSS_e_DE_ICRS,SDSS_umag,SDSS_gmag,SDSS_rmag,SDSS_imag,SDSS_zmag,SDSS_e_umag,SDSS_e_gmag,SDSS_e_rmag,SDSS_e_imag,SDSS_e_zmag,SDSS_zsp,SDSS_e_zsp,SDSS_f_zsp,SDSS_spCl,SDSS_subCl,SDSS_pmRA,SDSS_pmDE,SDSS_e_pmRA,SDSS_e_pmDE,SDSS_sigRA,SDSS_sigDE,SDSS_zph,SDSS_e_zph,SDSS_<zph>,SDSS_Q,SDSS_SDSS16,SDSS_Sp-ID,SDSS_MJD,MILQ_angDist,MILQ_RA,MILQ_DEC,MILQ_NAME,MILQ_TYPE,MILQ_RMAG,MILQ_BMAG,MILQ_COMMENT,MILQ_R,MILQ_B,MILQ_Z,MILQ_CITE,MILQ_ZCITE,MILQ_RXPCT,MILQ_QPCT,MILQ_XNAME,MILQ_RNAME,MILQ_LOBE1,MILQ_LOBE2,class_GAIA_class,class_SDSS_class,class_MILQ_class,class_final
0,SRGe J101903.3+563020,154.763934,56.505585,154.966043,50.066118,9.267930,15.858959,2.171120e-14,5.650445e-15,28.800014,7.495345,1211.871216,0.0,0.0,0.0,6207,5146,154.765943,56.504295,4.058504,102.778427,0.000023,6.268413e+08,6.270701e+08,0.0,0.0,0.023765,0.006185,7.077317,154.763934,56.505585,853324209744036352,7.576342,1,0,20.950485,-1.0,0,SDSS J101902.92+563016.4,0.949746,QSO,7.576898,1,0,0,2,16.915001,16.554001,12.382,12.1,5.0,0.7,2,1,1,1237658303814435106,921673331

In [35]:

def assign_final_class(row):
    if not pd.isna(row.class_MILQ_class):
        return row.class_MILQ_class
    elif not pd.isna(row.class_SDSS_class):
        return row.class_SDSS_class
    elif not pd.isna(row.class_GAIA_class):
        return row.class_GAIA_class
    else:
        return 'unknown'

def assign_final_redshift(row):
    if not pd.isna(row.MILQ_Z):
        return row.MILQ_Z
    elif not pd.isna(row.SDSS_zsp):
        return row.SDSS_zsp
    elif not pd.isna(row.SDSS_zph):
        return row.SDSS_zph
    else:
        return np.nan

desi_matched.loc[:, 'class_final'] = desi_matched.apply(assign_final_class, axis=1)
desi_matched.loc[:, 'redshift_final'] = desi_matched.apply(assign_final_redshift, axis=1)


num_qso = len(desi_matched.query('class_MILQ_class=="QSO"'))
num_qso += len(desi_matched.query('class_SDSS_class=="QSO" & class_MILQ_class.isna()'))
num_qso += len(desi_matched.query('class_SDSS_class.isna() & class_MILQ_class.isna() & class_GAIA_class=="QSO"'))
desi_matched.class_final.value_counts()['QSO'] == num_qso, f'QSO count is wrong, should be {num_qso}, got {desi_matched.class_final.value_counts()["QSO"]}'
desi_matched.class_final.value_counts()

unknown    6113
QSO        2580
STAR        466
GALAXY      341
Name: class_final, dtype: int64

In [37]:
desi_matched.to_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14_desi_nway_match_xmatch.pkl')